In [37]:
import numpy as np
class RKAS:
    def __init__(self, tol=1e-4, max_iter=1000):
        """
        Randomized Kaczmarz with Adaptive Stepsizes (RKAS).
        :param tol: Convergence tolerance.
        :param max_iter: Maximum number of iterations.
        """
        self.tol = tol
        self.max_iter = max_iter
        self.alpha = 0.99
        self.epsilon = 1e-8

    def solve(self, A, b, x):
        """
        Solves the inconsistent system Ax = b using RKAS.
        :param A: (m x n) coefficient matrix.
        :param b: (m x 1) right-hand side vector.
        :return: Approximate solution x.
        """
        m, n = A.shape
        x = np.zeros((n, 1))  # Initial solution x^0 = 0
        r = -b  # Initial residual r^0 = Ax^0 - b = -b

        # # Compute row selection probabilities
        # row_norms = np.linalg.norm(A, axis=1) ** 2
        # probabilities = row_norms / np.sum(row_norms)  # Pr(i_k = i)

        for k in range(self.max_iter):
                       # Compute exponential weighting for the rows
            row_norms = np.linalg.norm(A, axis=1)**2

            # Normalize by subtracting the maximum (robust to large values)
            row_norms -= np.max(row_norms)

            # Scale by alpha (can be adjusted for better performance)
            row_norms *= self.alpha

            # Add epsilon to prevent division by zero or log(0)
            row_norms += self.epsilon
            
            #Absolute value of row norm
            row_norms = np.abs(row_norms)

            # Calculate probabilities (should be stable now)
            probabilities = row_norms / np.sum(row_norms)

            # Ensure probabilities sum to 1 (handle potential rounding errors)
            probabilities /= np.sum(probabilities)

            # Step 1: Select row index i_k with probability Pr(i_k = i)
            i_k = np.random.choice(m, p=probabilities)

            # Extract row A_{i_k,:} and corresponding residual component
            A_ik = A[i_k, :].reshape(1, -1)  # Ensure row vector
            AAT_ik = np.dot(A, A_ik.T)  # Compute A * A^T for row i_k

            # Step 2: Compute adaptive step size
            alpha_k = np.dot(AAT_ik.T, r) / np.linalg.norm(AAT_ik) ** 2

            # Step 3: Update solution and residual
            x = x - alpha_k * A_ik.T
            r = r - alpha_k * AAT_ik

            # Check for convergence
            if np.linalg.norm(r) < self.tol:
                print(f"Converged in {k} iterations")
                break

        return x


In [38]:


# Function to generate a consistent overdetermined system (A * x = b)
def generate_consistent_system(m=13, n=2):
    """
    Generates a consistent overdetermined system Ax = b where:
    - A is an (m x n) random matrix
    - x_true is a random (n x 1) vector
    - b is computed as A @ x_true, ensuring consistency.
    """
    np.random.seed()  # Ensures reproducibility
    A = np.random.randn(m, n)  # Random 13x2 matrix
    x_true = np.random.randn(n, 1)  # True solution (random 2x1 vector)
    b = np.dot(A, x_true)  # Compute b such that Ax = b is consistent
    print("Condition number of A (Consistent):", np.linalg.cond(A))
    return A, b, x_true
    
# Function to generate a random inconsistent overdetermined system (A * x ≈ b)
def generate_inconsistent_system(m=13, n=2):
    """
    Generates a random overdetermined system Ax ≈ b where:
    - A is an (m x n) random matrix
    - b is a random (m x 1) vector, making the system potentially inconsistent.
    """
    np.random.seed()  # Ensures reproducibility

    A = np.random.randn(m, n)  # Random 13x2 matrix
    b = np.random.randn(m, 1)  # Random 13x1 vector (not necessarily from A*x)
    x_true = np.linalg.pinv(A) @ b  # Least-squares solution for reference
    print("condition number of A: ", np.linalg.cond(A))
  
    return A, b, x_true
# Choose which system to test
test_type = "inconsistent"  # Change to "inconsistent" or "consistent"
if test_type == "inconsistent":
    A, b, x_true = generate_inconsistent_system()
    x_true = np.linalg.pinv(A) @ b  # Least-squares solution
elif test_type == "consistent":
    A, b, x_true = generate_consistent_system()


rk_adaptive = RKAS()
# Initial guess (zero vector)
x0 = np.zeros((A.shape[1], 1))  # 2x1 zero vector

# Solve using RK method
num_iterations = 1000
tol = 1e-4

x_adaptive_approx= rk_adaptive.solve(A, b, x0)
# Print results
print("\nRK adaptive approximated solution (x_adaptive_approx):\n", x_adaptive_approx)
print("True solution (x_true):\n", x_true)
print("Error (||Ax - b|| using RK_adaptive):", np.linalg.norm(A @ x_adaptive_approx - b))
print("Error (||Ax - b|| using True Solution):", np.linalg.norm(A @ x_true - b))


condition number of A:  1.249682668470513

RK adaptive approximated solution (x_adaptive_approx):
 [[0.79539992]
 [0.12685113]]
True solution (x_true):
 [[0.79539992]
 [0.12685113]]
Error (||Ax - b|| using RK_adaptive): 4.072963361123535
Error (||Ax - b|| using True Solution): 4.072963361123535
